In [1]:
import sys
sys.path.append('../')

import numpy as np
import core
from core.shapes import Circle
from utils.utils import grad_num

%load_ext autoreload
%autoreload 2

In [2]:
test = np.array([1, 2, 3])
print(test[(slice(0, 2))])

[1 2]


In [3]:
def test_area(r):
    circle = Circle(r=r)
    return circle.area

In [4]:
test_area(1)

array([3.14159265])

In [5]:
core.set_backend('autograd')

In [6]:
def test_area(r):
    circle = Circle(r=r)
    return circle.area
test_area(1)

array([3.14159265])

In [7]:
from autograd import grad
import autograd.numpy as npa

In [8]:
grad(test_area)(1.0)

6.283185307179587

In [9]:
grad_num(test_area, np.array(1.0))

array(6.28318563)

In [10]:
# Initialize a lattice
lattice = core.Lattice('square')

def test_eps(rs):
    # Initialize a PhC (by default with eps = 1 in upper and lower cladding, we set upper one to 5)
    phc = core.PhotCryst(lattice, eps_u = 5)
    # Add a layer to the PhC with thickness 1 and background permittivity 10
    phc.add_layer(d=1, eps_b=10)
    for r in rs:
        circle = Circle(r=r)
        phc.layers[-1].add_shape(circle)
    
    return phc.layers[-1].eps_avg

print(test_eps([0.2]), test_eps([0.2, 0.2]))
print(type(test_eps([0.2])))

[8.86902664] [7.88017559]
<class 'numpy.ndarray'>


In [11]:
print(grad(test_eps)(np.array([0.2, 0.3])), grad_num(test_eps, np.array([0.2, 0.3])))
print(type(grad(test_eps)([0.2, 0.3])))

[ -8.11198173 -14.83276578] [ -8.11198375 -14.83276826]
<class 'list'>


In [12]:
def test_lattice(a1):
    lattice = core.Lattice(a1, npa.array([0., 1.]))
    return lattice.bz_area

print(test_lattice([.5, 0.]))
print(type(test_lattice([.5, 0.])))

78.95683520871486
<class 'numpy.float64'>


In [13]:
print(grad_num(test_lattice, npa.array([0.2, 0.3])))
print(grad(test_lattice)(npa.array([0.2, 0.3])))
# print(type(grad(test_lattice)(npa.array([0.2, 0.3]))))

[-986.95994666    0.        ]
[-986.96044011    0.        ]


/home/momchil/anaconda3/lib/python3.7/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


In [14]:
print(grad(test_lattice)(np.array([0.2, 0.3])))

[-986.96044011    0.        ]


/home/momchil/anaconda3/lib/python3.7/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


In [8]:
def test_pwe(r):
    # Initialize a lattice
    lattice = core.Lattice('square')
    # Initialize a layer
    layer = core.Layer(lattice, eps_b = 1)
    # Add a shape to this layer 
    layer.add_shape('circle', {'x': 0, 'y': 0.1, 'r': r, 'eps': 8.9})
    pwe = core.PlaneWaveExp(layer, gmax=10)
    
    return npa.max(npa.imag(pwe.T1[1]))

print(test_pwe([0.2]))
print(grad(test_pwe)(np.array(0.2)), grad_num(test_pwe, np.array(0.2)))

0.47567186901682823
3.749184159165697 3.7491841586856367


In [10]:
def test_pwe(r):
    # Initialize a lattice
    lattice = core.Lattice('square')
    # Initialize a layer
    layer = core.Layer(lattice, eps_b = 1)
    # Add a shape to this layer 
    layer.add_shape('circle', {'x': 0, 'y': 0.1, 'r': r, 'eps': 8.9})
    pwe = core.PlaneWaveExp(layer, gmax=10)
    pwe.compute_eps_inv()
    
    return npa.real(npa.sum(pwe.eps_inv_mat))

print(test_pwe([0.2]))
print(grad(test_pwe)(np.array(0.2)), grad_num(test_pwe, np.array(0.2)))

55.448120372278865
-65.47691732809798 -65.47693629954665


In [15]:
def test_pwe(r):
    # Initialize a lattice
    lattice = core.Lattice('square')
    # Initialize a layer
    layer = core.Layer(lattice, eps_b = 1)
    # Add a shape to this layer 
    layer.add_shape('circle', {'x': 0, 'y': 0.1, 'r': r, 'eps': 8.9})
    pwe = core.PlaneWaveExp(layer, gmax=10)
    pwe.run(pol='tm')
    
    return pwe.freqs[2]

print(test_pwe([0.2]))
print(grad(test_pwe)(np.array(0.2)), grad_num(test_pwe, np.array(0.2)))

0.6278853353619951
-2.4519325208774907 -2.451931510050187


To figure out for PWE:
- how to store eigenvalues over multiple k-s
- eigenvector problem seems to be for k = 0
- check out what broadcasting is about for eigh
- implement polygonal shapes...
- check out if pytorch or some other package supports eigenvalues

For autograd eigh:
- Write a test for the eigenvalue only part for hermitian matrices
- Write pull request...
